# Import libraries

In [1]:
import pandas as pd
import sys
import numpy as np
import random

# Import data

In [164]:
df1 = pd.read_excel('Dane_S2_50_10.xlsx')
df1 = df1.drop('Zadanie',1)

df2 = pd.read_excel('Dane_S2_100_20.xlsx')
df2 = df2.drop('Zadanie',1)

df3 = pd.read_excel('Dane_S2_200_20.xlsx')
df3 = df3.drop('Zadanie',1)
df3.head()

,M1,M2,M3,M4,M5,M6,M7,M8,M9,M10,M11,M12,M13,M14,M15,M16,M17,M18,M19,M20
0,83,47,75,13,97,5,99,42,77,74,89,52,37,4,94,26,18,68,79,26
1,94,54,98,59,2,91,65,70,2,16,97,7,71,73,24,65,68,26,49,43
2,45,84,2,32,95,71,45,87,18,88,1,77,68,56,98,92,87,80,63,63
3,44,82,14,22,40,97,35,86,67,2,77,29,90,2,6,54,65,17,1,14
4,35,96,37,94,81,20,11,62,11,43,90,8,51,60,7,52,53,81,24,66


# Functions

In [167]:
def CountScore(df):
    m_czasy = []
    v_czasy = []

    for i in range(len(df)):
        m_czasy.append([0] * len(df.columns))

    for i in range(len(df)):
        for j in range(len(df.columns)):
            if(i!=0 and j!=0):
                czas = max( m_czasy[i-1][j] + df.iloc[i,j],m_czasy[i][j-1] + df.iloc[i,j]) 
                m_czasy[i][j] = czas
            elif(i==0 and j==0):
                czas = df.iloc[i,j]
                m_czasy[i][j] = czas
            elif(i==0):
                czas = max(df.iloc[i,j],m_czasy[i][j-1] + df.iloc[i,j])
                m_czasy[i][j] = czas
            elif(j==0):
                czas = max(df.iloc[i,j],m_czasy[i-1][j] + df.iloc[i,j])
                m_czasy[i][j] = czas

    return m_czasy[i][j]

def Cross(assigment1,assigment2,idx=5):
# def Cross(df1,df2,idx=5):
#     idx=random.randrange(0,len(df1),1)
#     print(idx)
    
#     a=df1[:idx], df1[idx:]
#     b=df2[:idx], df2[idx:]
#     child1_part1 = list(a[0].index) 
#     child1_part2 = list (a[1].index)
    
#     child2_part1= list(b[0].index) 
#     child2_part2 = list(b[1].index) 
    a=assigment1[:idx], assigment1[idx:]
    b=assigment2[:idx], assigment2[idx:]
    
    child1_part1 = a[0]
    child1_part2 = a[1]
    child2_part1 = b[0]
    child2_part2 = b[1]

    child1 = child1_part1 + [ x for x in child2_part2 if not x in child1_part1]
    child1 = child1 + [ x for x in child1_part2 if not x in child1]
    
    child2 = child2_part1 + [ x for x in child1_part2 if not x in child2_part1]
    child2 = child2 + [ x for x in child1_part1 if not x in child2]
    
#     return df1.iloc[child1] df1.iloc[child2]
    return child1, child2
    
# def Mutation():
    #simple swap
    
# def CrossAlt():
    
# def MutationAlt():
    #inwersja

# def Tournament():

#Cross([1,2,3,4,5,6,7,8,9,10], [10,9,8,7,6,5,4,3,2,1])

([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], [10, 9, 8, 7, 6, 1, 2, 3, 4, 5])

In [59]:
# c1 = Cross(df, df.sample(n=len(df)))[1]
# c1

In [169]:
df = df1[:10]

In [173]:
list(df.index)



[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

# Notes

In [6]:
# abc = [11,12,13]
# abc + abc
# len([5, 1, 2]+[1,2])
# [ x for x in list(Cross(df,df,5)[1].index) if not x in abc]
# Cross(df2, df2.sample(n=len(df2)), 5)[0]
# df1.sample(n=len(df1))
# Cross(df, df.sample(n=len(df)), 5)[0]